In [7]:
import requests
import pandas as pd

# Define the API URL
api_url = "https://api.datacite.org/dois"

# Define the query parameters
params = {
    "resource-type-id": "Event",
    "page[number]": 1,
    "page[size]": 1000
}

# Initialize lists to store data
dois = []
publication_years = []

# Function to fetch data from API and append to lists
def fetch_data(url, dois, publication_years):
    response = requests.get(url, params=params)
    data = response.json()
    for item in data["data"]:
        attributes = item["attributes"]
        dois.append(attributes["doi"])
        publication_years.append(attributes["publicationYear"])
    return data["links"]["next"]

# Fetch data from first page
next_url = fetch_data(api_url, dois, publication_years)

# Fetch data from subsequent pages until 2 pages are fetched or no more data is available
page_count = 1
while next_url and page_count < 2:
    next_url = fetch_data(next_url, dois, publication_years)
    page_count += 1

# Create a DataFrame from the extracted data
df = pd.DataFrame({
    "DOI": dois,
    "Publication Year": publication_years
})

# Generate a statistic of events per publication year
event_counts = df["Publication Year"].value_counts().sort_index()

# Print the statistic
print("Statistic of events per publication year:")
print(event_counts)


Statistic of events per publication year:
1987      2
1991      2
1994      2
1997      2
1999      2
2000     10
2001      2
2002      2
2003      6
2004     10
2005     10
2006     12
2007      6
2008     12
2009     12
2010     12
2011     18
2012     18
2013     10
2014     12
2015     22
2016     36
2017     84
2018     60
2019    132
2020     94
2021    102
2022    132
2023    192
2024    974
2025     10
Name: Publication Year, dtype: int64


In [3]:
print(df)

                   DOI  Publication Year
0   10.25798/33aq-mc61              2024
1   10.25798/s3qh-cq80              2022
2   10.25798/5s6f-hh60              2024
3   10.25798/wh3v-dc90              2023
4   10.25798/3re7-zn50              2024
5   10.25798/zn0h-5n04              2024
6   10.25798/k1jj-pf70              2023
7   10.25798/jcr1-pz83              2000
8   10.25798/ygr9-2994              2001
9   10.25798/ky5d-rw07              2002
10  10.25798/6mss-1a52              2003
11  10.25798/9fn0-ry77              2007
12  10.25798/t0j3-1n96              2004
13  10.25798/y62w-c022              2005
14  10.25798/wybk-n755              2006
15  10.25798/9wrc-5j18              2008
16  10.25798/b3ts-2376              2009
17  10.25798/8xt6-j753              2010
18  10.25798/wrc6-9q32              2011
19  10.25798/eqqv-rx65              2012
20  10.25798/x32a-5w82              2013
21  10.25798/apwa-hn93              2014
22  10.25798/p94j-jv21              2015
23  10.25798/z1y